# Converting a Normal EPANET .inp File to a FCV-Res IWS File [1]
This notebook takes an input EPANET files with demands input normally as a CWS base demand and adds the necessary elements to convert it into a FCV-Reservoir (FCVRes) IWS simulation  
For details on the FCVRes method refer to [1]   
A simplified schematic of the modified demand node in FCVRes is seen below:  

![](FCVRes.png)


### First, we import the necessary libraries and packages
**WNTR** for building EPANET network models in Python  
**NUMPY & PANDAS** for data handling and processing  
**re** for searching and matching text in the .inp file using regular expressions

In [23]:
import wntr  
import numpy as np 
import pandas as pd
import re
import pathlib

### Specifying paths for simulation files and preprocessing the input
**Warning:** *Paths in this script (and the rest of this repo) are absolute unless running the network files provided within the repo*  
Input filename (with extensions) as string.  
For running the .inp files in this repository, you can use this relative path `"../Network-Files/Network X/"` where X is the network number 

In [24]:
 # Replace with appropriate path and filename
directory=pathlib.Path("../Network-Files/Network 3")
filename=pathlib.Path("Network3_4hr_PDA.inp")
name_only=str(filename.stem)[0:-4]
print("Selected File: ",name_only)
abs_path=directory/filename

Selected File:  Network3_4hr_


### Necessary Assumptions Input
Converting a CWS demand-driven analysis into an IWS pressure-driven analysis requires some assumptions in all methods  
The resistance of the service connection between the demand node and the household (end-user) is uncertain and is modelled using two assumptions  
The **desired head (pressure)** is the pressure at the demand node at which (or above) the consumer can satisfy their full demand in the supply duration (or possible less)  
The **minimum head (pressure)** is the minimum pressure at the demand node required for flow to begin passing through the service connection  
These two assumptions dictate the flow-pressure relationship that determines the pressure-dependent flow through the service connection as follows:

$$ Q\, = \!Q_{des}(\frac{H_{j}-H^{min}}{H^{des}-H^{min}})^{\frac{1}{n_j}} \quad[1]$$ 
Where Q is the flow through the service connection, $Q_{des}$ is the desired (base) demand, $H_j$ is the head at the demand node $j$, $H^{min}$ is the minimum head, and $H^{des}$ is the desired head


In [25]:
desired_pressure=10     # Set the desired pressure
minimum_pressure=0      # Set the minimum pressure
### Nj=0.5
pressure_diff=desired_pressure-minimum_pressure  

### Extracting information from the input file
To modify the .inp file, demand node IDs, elevations, x and y coordinates  
We use wntr to build the network model of the input file and use wntr's junctions module to extract the details of each node

In [26]:
demand_nodes=[]       # For storing list of nodes that have non-zero demands
desired_demands=[]    # For storing demand rates desired by each node for desired volume calculations
elevations=[]         # For storing elevations of demand nodes
xcoordinates=[]       # For storing x coordinates of demand nodes
ycoordinates=[]       # For storing y coordinates of demand nodes
all_nodes=[]          # For storing list of node ids of all nodes
all_elevations=[]     # For storing elevations of all nodes

# Creates a network model object using EPANET .inp file
network=wntr.network.WaterNetworkModel(abs_path)

# Iterates over the junction list in the Network object
for node in network.junctions():
    all_nodes.append(node[1].name)
    all_elevations.append(node[1].elevation)
    # For all nodes that have non-zero demands
    if node[1].base_demand != 0:
        # Record node ID (name), desired demand (base_demand) in CMS, elevations, x and y coordinates
        demand_nodes.append(node[1].name)
        desired_demands.append(node[1].base_demand)
        elevations.append(node[1].elevation)
        xcoordinates.append(node[1].coordinates[0])
        ycoordinates.append(node[1].coordinates[1])

# Get the supply duration in minutes (/60) as an integer
supply_duration=int(network.options.time.duration/60)

### Writing Reservoirs Section
For every demand node, an artificial reservoir (AR) will be added at an elevation of $E_{reservoir}=E_{Demand\,Node}+H^{min}$  
Raising the elevation by $H^{min}$ enforces the minimum pressure threshold as no flow to the user would occur if the pressure at the node is less than that value  

In [27]:
# Adds "AR" to each demand node id to be used as ID for AR
reservoirids=["AR"+str(id) for id in demand_nodes]
# Calculates the elevation of the AR
reservoir_elevs=[elevation + minimum_pressure for elevation in elevations ]
# No Patterns are assigned to any of the ARs
reservoir_patterns=["    "]*len(reservoirids)
# Semicolons to end each line
semicolons=[";"]*len(reservoirids)
# Dataframe with all the required fields for AR [ID   Elevation   Pattern   ;]
added_reservoirs=pd.DataFrame(list(zip(reservoirids,reservoir_elevs,reservoir_patterns,semicolons)))
# Exports the added reservoirs as a list of strings where each entry is a line of the reservoirs section
added_reservoirs=added_reservoirs.to_string(header=False,index=False).splitlines()

### Writing Artificial Nodes
As seen in the figure above, each demand nodes will require the addition of an Artificial Nodes (AN) to connect the FCV to the AR  
The artificial node for Node X (will be named ANforNodeX) will have the same elevation as Node X  

In [28]:
# Adds the phrase "AN1forNode" to each node id as the id of the first artificial node (AN) added to each demand node
anodeids=["ANforNode"+str(id) for id in demand_nodes]
# Sets the base demand for all added nodes as 0
base_demands=[0]*len(anodeids)
# No demand pattern is assigned to any demand node
demand_patterns=["     "]*len(anodeids)
# Dataframe with all the required fields for AN1 [ID   Elevation   Demand   Pattern   ;]
added_nodes=pd.DataFrame(list(zip(anodeids,elevations,base_demands,demand_patterns,semicolons)))
# Exports the added junctions as a list of strings where each entry is a line of the junctions section
added_nodes=added_nodes.to_string(header=False,index=False,col_space=10).splitlines()

### Writing Added Pipes
Similar to the reservoirs section, each field for the pipe entries is stored in a separate list  
FCVRes method requires the addition of one pipe (Refer to the above figure)  
The pipe should have a negligible major loss and thus will have a small length (0.1 m), a diameter of 350 mm, a HW coefficient of 130, and a minor loss coefficient of:  
$$ K_{Minor}=(H^{des}-H^{min})\frac{g\pi^2D^4}{8Q_{des}^2}$$
The diameter is unified at 0.35 m for simplicity (350 mm)  
While the second pipe will have a minor loss coefficient of 0, and serves to connect the PSV to the downstream artificial reservoir  


In [29]:
# Adds the phrase PipeforNode to each node id and stores it as a pipe id
pipeids=['Pipe1forNode'+str(id) for id in demand_nodes]
# Calculates the minor loss coefficient of each pipe to simulate the head-flow relationship
minorloss=[pressure_diff*9.81*np.pi**2*0.35**4/(8*demand**2) for demand in desired_demands]
# Sets all lengths to 0.1 m
lengths=[0.1]*len(pipeids)
# Sets all diameters to 1 m (1000 mm)
diameters_pipes=[350]*len(pipeids)
# Sets all Hazen-Williams Coefficients as 130
hazen=[130]*len(pipeids)
# Sets all created pipes to work as Check Valved to prevent backflow
status=['CV']*len(pipeids)
# list of semicolons
semicolons=[";"]*len(pipeids)
# Assemble all lists into a dataframe where each row is the definition for one simple reservoir
# Data frame with all required fields [ID   Node1   Node2   Length   Diameter   Roughness   MinorLoss   Status   ;]
added_pipes=pd.DataFrame(list(zip(pipeids,anodeids,reservoirids,lengths,diameters_pipes,hazen,minorloss,status,semicolons)))
# Exports the pipe section as a list of strings where each entry is a line of the pipes section
added_pipes=added_pipes.to_string(header=False,index=False,col_space=10).splitlines()

### Writing Valves Section
For each original demand node, a Flow-Control Valve is added restrict the flow to the users desired flow rate $Q^{des}$  

In [30]:
# Adds the phrase APSVforNode to each node id and stores it as a PSV valve id
valveids=["FCVforNode"+str(id) for id in demand_nodes]
# From nodes are the original demand nodes and to nodes are the artificial nodes
# Sets all valve diameters to 12 (will not affect head loss across valve)
valve_diameters=[12.0000]*len(valveids)
# Sets the type of all valves to Flow-Control Valves
valve_types=["FCV"]*len(valveids)
# Sets the valve setting for each valve to the base demand of the original demand nodes (converts back to LPS)
valve_settings=[demand*1000 for demand in desired_demands]
# Sets the minor loss coefficient across the valve to 0
valve_minor_loss=["0.0000"]*len(valveids)
# Semicolons at the end of each line
semicolons=[';']*len(valveids)
# Data frame with all required fields [ID   Node1   Node2   Diameter   Type   Setting   MinorLoss   ;]
added_valves=pd.DataFrame(list(zip(valveids,demand_nodes,anodeids,valve_diameters,valve_types,valve_settings,valve_minor_loss,semicolons)))
added_valves=added_valves.to_string(header=False,index=False,col_space=10).splitlines()

### Writing Reservoir & AN Coordinates  
Each reservoir's coordinates are translated by XX m in both x and y directions  
The ANs will be translated YY meters in both directions  
These coordinates are merely for display and do not affect the simulation in any way  
[SCALE THESE VALUES FOR BETTER DISPLAY BASED ON YOUR INPUT FILE SCALE]

In [31]:
# Set preferred translation distance for [AN1,AN2,AT] where AN is Artificial Node and AT is the Artifical Tank
x_direct_distance=[-30,-60]
y_driect_distance=[30,0]
# Translates the reservoirs by a 100 m in both axes 
anode_xcoord=[x+x_direct_distance[0] for x in xcoordinates]
reservoir_xcoord =[x+x_direct_distance[1] for x in xcoordinates]
anode_ycoord=[y+y_driect_distance[0] for y in ycoordinates]
reservoir_ycoord =[y+y_driect_distance[1] for y in ycoordinates]

added_xcoordinates=anode_xcoord+reservoir_xcoord
added_ycoordinates=anode_ycoord+reservoir_ycoord
ids_coords=anodeids+reservoirids

# Assemble all lists into a dataframe where each row is the coordinates for one artificial reservoir or node
added_coordinates=pd.DataFrame(list(zip(ids_coords,added_xcoordinates,added_ycoordinates)))
# Exports the coordinate section as a list of strings where each entry is a line of the coordinates section
added_coordinates=added_coordinates.to_string(header=False,index=False,col_space=10).splitlines()

### Rewriting Base Demands
After creating the required additions to the original demand nodes, the base demands for all demand junctions has to be reset to zero

In [32]:
# List of zero base demands for all nodes
zerodemands=[0]*len(all_nodes)
# White space indicating no patterns
pattern=['     ']*len(all_nodes)
semicolons=[';']*len(all_nodes)
original_nodes=pd.DataFrame(list(zip(all_nodes,all_elevations,zerodemands,pattern,semicolons)))
original_nodes=original_nodes.to_string(header=False,index=False,col_space=10).splitlines()

In [33]:
# opens .inp file to read
file=open(abs_path,'r')
lines=[]            # list to store all lines in the .inp file
linecount=0         # Counter for the number of lines
junctions_marker=0  # To store the line number at which the junctions section starts
reservoirs_marker=0 # To store the line number at which the reservoir section ends
pipes_marker=0      # To store the line number at which the pumps section starts
valves_marker=0     # to store the line number at which the valves section
coords_marker=0     # To store the line number at which the vertices section starts

# Loops over each line in the input file 
for line in file:
    # Record the position of the phrase [JUNCTIONS] and add 2 to skip the header line
    if re.search('\[JUNCTIONS\]',line):
        junctions_marker=linecount+2
    # Record the position of the phrase [TANKS] and subtract 1 for the end of the reservoirs section
    if re.search('\[TANKS\]',line):
        reservoirs_marker=linecount-1
     # Record the position of the phrase [PUMPS] and subtract 1 to add pipes to the end of the pipe section
    if re.search('\[PUMPS\]',line):
        pipes_marker=linecount-1
    # Record the position of the phrase [VALVES] and add 2 to skip the header line
    if re.search('\[VALVES\]',line):
        valves_marker=linecount+2
     # Record the position of the phrase [Vertices] and subtract 1 to add Tank cooridnates to the end of the coordinates section
    if re.search('\[VERTICES\]',line):
        coords_marker=linecount-1
    linecount+=1
    # Store all lines in a list
    lines.append(line)
file.close()

# Translate the reservoirs marker by the length of the added nodes (ANs) that will be added before it (as it will displace all subsequent lines)
reservoirs_marker+=len(added_nodes)
# Translate the pipes marker by the length of the reservoir section and the added nodesthat will be added before it (as it will displace all subsequent lines)
pipes_marker+=len(added_reservoirs)+len(added_nodes)
# Translate the coordinates marker by the length of the added reservoirs, pipes and nodes
valves_marker+=len(added_reservoirs)+len(added_pipes)+len(added_nodes)
# Translate the coordinates marker by the length of the added tanks, pipes, nodes and valves
coords_marker+=len(added_reservoirs)+len(added_pipes)+len(added_valves)+len(added_nodes)

# Inserts the created sections in their appropriate location in the list of lines
lines[junctions_marker:junctions_marker+len(original_nodes)]=original_nodes
lines[junctions_marker+len(original_nodes):junctions_marker+len(original_nodes)]=added_nodes
lines[reservoirs_marker:reservoirs_marker]=added_reservoirs
lines[pipes_marker:pipes_marker]=added_pipes
lines[valves_marker:valves_marker]=added_valves
lines[coords_marker:coords_marker]=added_coordinates

# Opens a new file in the same directory to write the modified network .inp file in
file=open(directory/pathlib.Path(name_only+'FCV-Res.inp'),'w')
c=0     #line counter

# All lines added by this script are missing a new line character at the end, the conditional statements below add the new line character for these lines only and writes all lines to the file
for line in lines:
    if c>=junctions_marker and c<=junctions_marker+len(original_nodes)+len(added_nodes):
        file.write(line+'\n')
    elif c>=reservoirs_marker and c<=reservoirs_marker+len(added_reservoirs):
        file.write(line+'\n')
    elif c>=pipes_marker and c<=pipes_marker+len(added_pipes):
        file.write(line+'\n')
    elif c>=valves_marker and c<=valves_marker+len(added_valves):
        file.write(line+'\n')
    elif c>=coords_marker and c<=coords_marker+len(added_coordinates):
        file.write(line+'\n')
    else: file.write(line)    
    c+=1
file.close()


### References:
##### [1] N. B. Gorev and I. F. Kodzhespirova, “Noniterative Implementation of Pressure-Dependent Demands Using the Hydraulic Analysis Engine of EPANET 2,” Water Resources Management, vol. 27, no. 10, pp. 3623–3630, Aug. 2013, doi: 10.1007/s11269-013-0369-1.